# Pertemuan I

Pada praktikum ini, Anda akan melakukan beberapa operasi dasar statistik dengan data bertema keseshatan yaitu data tumor.

* [Histogram](#1)
* [Outliers](#2)
* [Box Plot ](#3)
* [Summary Statistics](#4)
* [CDF](#5)
* [Effect size](#6)
* [Relationship Between Variables](#7)
* [Correlation](#8)
* [Covariance](#9)
* [Pearson Correlation](#10)
* [Spearman's Rank Correlation](#11)
* [Mean VS Median](#12)
* [Hypothesis Testing](#13)
* [Normal(Gaussian) Distribution and z-score](#14)


# Instruksi Tugas Praktikum FIKTI dan FTI

1. Silahkan modifikasi kode operasi yang ada menggunakan library perhitungan berbasis GPU (Library Cupy)
2. Jelaskan apakah terdapat korelasi antara variabel atau fitur compactness_mean dengan concavity_mean
3. Buatlah diagram korelasi dari fitur compactness_mean dengan concavity_mean
4. Hitung nilai covariance dari fitur compactness_mean dengan concavity_mean. Jelaskan arti dari nilai tersebut.
5. Hitung nilai pearson correlation dari fitur compactness_mean dengan concavity_mean. Jelaskan arti dari nilai tersebut.
6. Lakukan uji hipotesis untuk kolom fitur compactness_mean dengan concavity_mean yang berbeda serta berikan penjelasan terhadap hasil dari uji hipotesis yang dilakukan.

# Instruksi Tugas Praktikum Prodi Mesin, FK, FTSP dan Farmasi

1. Lakukan perhitungan summary nilai statistik (mean, standar deviasi, dan variance) dari fitur texture_mean
2. Berdasarkan hasil ploting diagram heatmap, jelaskan korelasi yang ada antara fitur compactness_se dan fractal_dimension_se
3. Hitung nilai pearson correlation dari fitur compactness_se dan fractal_dimension_se. Jelaskan arti dari nilai tersebut.
4. Lakukan uji hipotesis untuk kolom fitur compactness_se dan fractal_dimension_se yang berbeda serta berikan penjelasan terhadap hasil dari uji hipotesis yang dilakukan.

In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
from scipy import stats
plt.style.use("ggplot")
import warnings
warnings.filterwarnings("ignore")
from scipy import stats

In [2]:
# read data as pandas data frame
url_data = "https://raw.githubusercontent.com/supasonicx/ATA-praktikum-01/main/data.csv"
data = pd.read_csv(url_data)
data = data.drop(['Unnamed: 32','id'],axis = 1)

### Overview Dataset

In [ ]:
## Melihat 5 baris awal dari dataset yang digunakan
data.head()

#### Note:
---
kolom/field diagnosis :
* M(malignant)   = Tumor Ganas
* Beningn(Jinak) = Tumor Jinak

In [ ]:
##
data.shape

In [ ]:
data.columns

<a id="1"></a> <br>
## Histogram

* Menampilkan Berapa kali (frekuensi) setiap nilai muncul dalam kumpulan data.
* Jenis deskripsi ini disebut distribusi variabel
* Cara paling umum untuk merepresentasikan distribusi variabel adalah histogram yaitu grafik yang menunjukkan frekuensi dari setiap nilai.
* Frequency = berapa kali setiap nilai muncu
* Contoh: [1,1,1,1,2,2,2]. Frequency dari 1 adalah empat dan frequency dari 2 adalah tiga.

In [ ]:
## ploting untuk data tumor dengan label ganas
m = plt.hist(data[data["diagnosis"] == "M"].radius_mean,bins=30,fc = (1,0,0,0.5),label = "Ganas")
## ploting untuk data tumor dengan label jinak
b = plt.hist(data[data["diagnosis"] == "B"].radius_mean,bins=30,fc = (0,1,0,0.5),label = "Jinak")
plt.legend()
plt.xlabel("Nilai Radius Mean")
plt.ylabel("Frequency")
plt.title("Histogram Radius Mean untuk tumor jinak dan ganas")
plt.show()
frequent_malignant_radius_mean = m[0].max()
index_frequent_malignant_radius_mean = list(m[0]).index(frequent_malignant_radius_mean)
most_frequent_malignant_radius_mean = m[1][index_frequent_malignant_radius_mean]
print("Rata-rata radius ganas yang paling sering adalah: ",most_frequent_malignant_radius_mean)

* Mari kita lihat kesimpulan lainnya
* Dari grafik ini terlihat bahwa rata-rata radius tumor ganas lebih besar dari rata-rata radius tumor jinak kebanyakan.
* Sebaran jinak (dalam grafik hijau) kira-kira berbentuk lonceng yaitu bentuk sebaran normal (distribusi gaussian)
* Anda juga dapat menemukan hasil seperti berapa rata-rata radius ganas yang paling sering

<a id="2"></a> <br>
## Outliers
* Saat melihat histogram seperti yang Anda lihat, ada nilai langka dalam distribusi jinak (grafik hijau)
* Nilai yang ada bisa berupa kesalahan atau kejadian langka.
* Kesalahan dan kejadian langka ini bisa disebut outlier.
* Menghitung outlier:
    * pertama kita perlu menghitung kuartil pertama (Q1)(25%)
    * kemudian temukan IQR(inter quartile range) = Q3-Q1
    * terakhir menghitung Q1 - 1.5*IQR dan Q3 + 1.5*IQR
    * Apa pun di luar kisaran ini adalah outlier
    * mari kita tulis kode untuk distribusi tumor jinak untuk rata-rata fitur radius

In [ ]:
data_jinak = data[data["diagnosis"] == "B"]
data_ganas = data[data["diagnosis"] == "M"]
desc = data_jinak.radius_mean.describe()
Q1 = desc[4]
Q3 = desc[6]
IQR = Q3-Q1
lower_bound = Q1 - 1.5*IQR
upper_bound = Q3 + 1.5*IQR
print("Apa pun di luar kisaran ini adalah outlier: (", lower_bound ,",", upper_bound,")")
data_jinak[data_jinak.radius_mean < lower_bound].radius_mean
print("Outliers: ",data_jinak[(data_jinak.radius_mean < lower_bound) | (data_jinak.radius_mean > upper_bound)].radius_mean.values)

<a id="3"></a> <br>
## Box Plot
* Anda dapat melihat outlier juga dari box plot
* Kita menemukan 3 outlier pada mean/rata-rata radius jinak dan pada box plot terdapat 3 outlier.

In [ ]:
melted_data = pd.melt(data,id_vars = "diagnosis",value_vars = ['radius_mean', 'texture_mean'])
plt.figure(figsize = (15,10))
sns.boxplot(x = "variable", y = "value", hue="diagnosis",data= melted_data)
plt.show()

<a id="4"></a> <br>
## Summary Statistics
* Mean/rata-rata
* Variance: penyebaran distribusi
* Standart deviation square root dari variance
* Mari kita lihat ringkasan statistik rata-rata pancaran tumor jinak:

In [ ]:
print("mean: ",data_jinak.radius_mean.mean())
print("variance: ",data_jinak.radius_mean.var())
print("standart deviation (std): ",data_jinak.radius_mean.std())
print("describe method: ",data_jinak.radius_mean.describe())

<a id="5"></a> <br>
## Cummulative Distribution Function
* Fungsi distribusi kumulatif adalah probabilitas bahwa variabel mengambil nilai kurang dari atau sama dengan x. P(X <= x)
* Mari kita jelaskan dalam grafik cdf mean radius jinak
* dalam grafik, berapa P(12 < X)? Jawabannya adalah 0,5. Probabilitas bahwa variabel mengambil nilai kurang dari atau sama dengan 12 (radius mean) adalah 0,5.
* Anda dapat memplot cdf dengan dua metode berbeda

In [ ]:
plt.hist(data_jinak.radius_mean,bins=50,fc=(0,1,0,0.5),label='Jinak',density=True, cumulative = True)
sorted_data = np.sort(data_jinak.radius_mean)
y = np.arange(len(sorted_data))/float(len(sorted_data)-1)
plt.plot(sorted_data,y,color='red')
plt.title('CDF dari radius mean/rata-rata tumor jinak')
plt.show()

<a id="6"></a> <br>
## Effect size
* Salah satu ringkasan statistik.
* Ini menggambarkan ukuran efek/effect size. Ini adalah cara sederhana untuk mengukur perbedaan antara dua kelompok.
* Dengan kata lain, ukuran efek menekankan ukuran perbedaan
* Gunakan ukuran efek kohen/cohen
* Cohen menyarankan bahwa jika d(ukuran efek)= 0,2, itu adalah ukuran efek kecil, d = 0,5 ukuran efek sedang, d = 0,8 ukuran efek besar.
* mari kita bandingkan ukuran efek antara mean radius jinak dan mean radius ganas
* Ukuran efek adalah 2.2 dan itu terlalu besar dan dikatakan bahwa dua kelompok itu berbeda satu sama lain seperti yang kita harapkan. Karena kelompok kita adalah mean radius jinak dan mean radius ganas yang berbeda satu sama lain

In [ ]:
mean_diff = data_ganas.radius_mean.mean() - data_jinak.radius_mean.mean()
var_bening = data_jinak.radius_mean.var()
var_malignant = data_ganas.radius_mean.var()
var_pooled = (len(data_jinak)*var_bening +len(data_ganas)*var_malignant ) / float(len(data_jinak)+ len(data_ganas))
effect_size = mean_diff/np.sqrt(var_pooled)
print("Effect size: ",effect_size)

<a id="7"></a> <br>
## Relationship Between Variables
* Kita dapat mengatakan bahwa dua variabel terkait satu sama lain, jika salah satunya memberikan informasi tentang yang lain
* Misalnya, harga dan jarak. Jika Anda pergi jarak jauh dengan taksi Anda akan membayar lebih. Oleh karena itu kita dapat mengatakan bahwa harga dan jarak berhubungan positif satu sama lain.
* Scatter Plot, Cara termudah untuk memeriksa hubungan antara dua variabel
* Mari kita lihat hubungan antara radius mean dan mean area
* Di scatter plot Anda dapat melihat bahwa ketika radius mean meningkat, mean area juga meningkat. Oleh karena itu, mereka berkorelasi positif satu sama lain.
* Tidak ada korelasi antara mean area dan dimensi fraktal se. Karena ketika mean area berubah, dimensi fraktal se tidak terpengaruh oleh peluang mean area

In [ ]:
plt.figure(figsize=(15, 10))
sns.jointplot(data=data, x="radius_mean", y="area_mean", kind="reg")
plt.show()

In [ ]:
# dan juga kita dapat melihat hubungan antara lebih dari 2 distribusi
sns.set(style = "white")
df = data.loc[:,["radius_mean","area_mean","fractal_dimension_se"]]
g = sns.PairGrid(df,diag_sharey = False,)
g.map_lower(sns.kdeplot,cmap="Blues_d")
g.map_upper(plt.scatter)
g.map_diag(sns.kdeplot,lw =3)
plt.show()

<a id="8"></a> <br>
## Correlation
* Kekuatan hubungan antara dua variabel
* Mari kita lihat korelasi antara semua fitur.

In [ ]:
f,ax=plt.subplots(figsize = (18,18))
sns.heatmap(data.corr(),annot= True,linewidths=0.5,fmt = ".1f",ax=ax)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.title('Correlation Map')
plt.savefig('graph.png')
plt.show()

* Matriks besar yang mencakup banyak angka
* Kisaran angka ini adalah -1 hingga 1.
* Arti dari 1 adalah dua variabel yang saling berkorelasi positif seperti mean radius dan mean area
* Arti dari nol adalah tidak ada korelasi antara variabel seperti rata-rata radius dan fractal dimension se
* Arti dari -1 adalah dua variabel berkorelasi negatif satu sama lain seperti rata-rata radius dan mean/rata-rata fractal dimension.Sebenarnya korelasi antara keduanya bukan -1, melainkan -0,3 tetapi idenya adalah jika tanda korelasi negatif berarti ada adalah korelasi negatif.

<a id="9"></a> <br>
## Covariance
* Covariance adalah ukuran kecenderungan dua variabel untuk bervariasi bersama-sama
* Jadi covarians dimaksimalkan jika dua vektor identik
* Covarians adalah nol jika mereka ortogonal.
* Covariance negatif jika mereka menunjuk ke arah yang berlawanan
* Mari kita lihat kovarians antara mean radius dan mean area. Kemudian lihat radius mean dan fractal dimension se


In [ ]:
np.cov(data.radius_mean,data.area_mean)
print("Covariance diantara radius mean dan area mean: ",data.radius_mean.cov(data.area_mean))
print("Covariance diantara radius mean dan fractal dimension se: ",data.radius_mean.cov(data.fractal_dimension_se))

<a id="10"></a> <br>
## Pearson Correlation
* Pembagian covarians dengan standar deviasi variabel
* Mari kita lihat korelasi pearson antara mean/rata-rata radius dan mean/rata-rata area
* Pertama mari kita gunakan metode .corr() yang sebenarnya kita gunakan pada bagian korelasi. Di bagian korelasi kami sebenarnya menggunakan korelasi pearson :)
* p1 dan p2 adalah sama. Di p1 kita menggunakan metode corr(), di p2 kita menerapkan definisi korelasi pearson (cov(A,B)/(std(A)*std(B)))
* Seperti yang kita harapkan korelasi pearson antara area_mean dan area_mean adalah 1 yang berarti bahwa mereka adalah distribusi yang sama
* Juga pearson correlation antara area_mean dan radius_mean adalah 0,98 yang berarti saling berkorelasi positif dan hubungan antar keduanya sangat tinggi.
* Untuk lebih jelas apa yang kami lakukan di bagian korelasi dan bagian korelasi pearson adalah sama.


In [ ]:
p1 = data.loc[:,["area_mean","radius_mean"]].corr(method= "pearson")
p2 = data.radius_mean.cov(data.area_mean)/(data.radius_mean.std()*data.area_mean.std())
print('Pearson correlation: ')
print(p1)
print('Pearson correlation: ',p2)

<a id="11"></a> <br>
## Spearman's Rank Correlation
* Pearson correlation bekerja dengan baik jika hubungan antara variabel linier dan variabel kira-kira normal. Tapi itu tidak kuat, jika ada outlier
* Untuk menghitung korelasi spearman, kita perlu menghitung peringkat dari setiap nilai


In [ ]:
ranked_data = data.rank()
spearman_corr = ranked_data.loc[:,["area_mean","radius_mean"]].corr(method= "pearson")
print("Spearman's correlation: ")
print(spearman_corr)

* Korelasi Spearman sedikit lebih tinggi dari korelasi pearson
    * Jika hubungan antar distribusi tidak linier, korelasi spearman cenderung lebih baik dalam memperkirakan kekuatan hubungan
    * Korelasi Pearson dapat dipengaruhi oleh outlier. Sehingga Korelasi Spearman lebih kuat.

<a id="12"></a> <br>
## Mean VS Median
* Kadang-kadang sebagai gantinya mean kita perlu menggunakan median. Saya akan menjelaskan mengapa kita perlu menggunakan median dengan sebuah contoh
* Misalkan ada 10 orang yang bekerja di sebuah perusahaan. Bos perusahaan akan menaikkan gaji mereka jika mean/rata-rata gaji mereka lebih kecil dari 5


In [ ]:
gaji = [1,4,3,2,5,4,2,3,1,500]
print("Mean/rata-rata dari gaji: ",np.mean(gaji))

* Rata-rata gaji adalah 52,5 jadi bos berpikir bahwa oooo saya memberi banyak gaji kepada karyawan saya. Dan tidak membuat kenaikan gaji mereka. Namun seperti yang Anda tahu ini tidak adil dan 500 (gaji) adalah outlier untuk daftar gaji ini.
* Median menghindari outlier

In [ ]:
print("Median dari gaji: ",np.median(gaji))

* Sekarang rata-rata gaji adalah 3 dan kurang dari 5 dan karyawan akan menaikkan gaji mereka dan mereka senang dan situasi ini adil :)

<a id="13"></a> <br>
## Hypothesis Testing
* Classical Hypothesis Testing / Pengujian Hipotesis Klasik
* Apa yang Anda perlu lakukan untuk menjawab pertanyaan berikut : "diberikan sampel dan efek nyata, berapa peluang melihat efek seperti itu secara kebetulan"
* Langkah pertama adalah mengukur ukuran efek nyata dengan memilih statistik uji. Pilihan alami untuk statistik uji adalah perbedaan mean/rata-rata antara dua kelompok.
* Langkah kedua adalah mendefinisikan hipotesis nol yaitu model sistem berdasarkan asumsi bahwa efek yang tampak tidak nyata. Hipotesis nol adalah jenis hipotesis yang digunakan dalam statistik yang menyatakan bahwa tidak ada signifikansi statistik dalam serangkaian pengamatan yang diberikan. Hipotesis nol adalah hipotesis yang orang mencoba untuk menyangkalnya. Hipotesis alternatif adalah hipotesis yang orang ingin mencoba untuk membuktikannya.
* Langkah ketiga adalah menghitung p-value yaitu probabilitas melihat efek nyata jika hipotesis nol benar. Misalkan kita memiliki uji hipotesis nol. Kemudian kita hitung nilai p. Jika nilai p kurang dari atau sama dengan ambang batas, kami menolak hipotesis nol.
* Jika p-value rendah, pengaruh tersebut dikatakan signifikan secara statistik artinya tidak mungkin terjadi secara kebetulan. Oleh karena itu kita dapat mengatakan bahwa efeknya lebih mungkin muncul pada populasi yang lebih besar.
* Mari kita coba contohkan. Hipotesis nol: dunia rata. Hipotesis alternatif: dunia itu bulat. Beberapa ilmuwan mulai menyangkal hipotesis nol. Ini akhirnya mengarah pada refleksi hipotesis nol dan penerimaan hipotesis alternatif.
* Contoh lainnya. "efek ini nyata" ini adalah hipotesis nol. Berdasarkan asumsi itu kami menghitung probabilitas efek yang tampak. Itu adalah nilai-p. Jika nilai p rendah, kami menyimpulkan bahwa hipotesis nol tidak mungkin benar.
* Sekarang mari kita buat contoh kita:
    * Saya ingin mengetahui apakah rata-rata radius dan rata-rata area terkait satu sama lain? Hipotesis nol saya adalah bahwa "hubungan antara rata-rata radius dan rata-rata area adalah nol pada populasi tumor'.
    * Sekarang kita perlu menyangkal hipotesis nol ini untuk menunjukkan bahwa mean/rata-rata radius dan mean/rata-rata area berhubungan. (walaupun sebenarnya kita telah mengetahui hasilnya berdasarkan analisa korelasi yang telah dilakukan sebelumnya)
    * mari kita cari nilai p (nilai probabilitas)

In [ ]:
statistic, p_value = stats.ttest_rel(data.radius_mean,data.area_mean)
print('p-value adalah: ',p_value)

* Hasil perhitungan P values/ Nilai P hampir mendekati nol, sehingga kita dapat menolak hipotesis nol. Penolakan hipotesis ini memiliki arti nilai rata-rata radius dan rata-rata area pada populasi tumor saling berpengaruh.

<a id="14"></a> <br>
## Normal(Gaussian) Distribution and z-score
* Disebut juga distribusi berbentuk lonceng
* Daripada membuat definisi formal dari distribusi gaussian, saya ingin menjelaskannya dengan sebuah contoh.
* Contoh klasik adalah gaussian IQ Score/score iq.
     * Di dunia katakanlah IQ rata-rata adalah 110.
     * Ada beberapa orang yang super cerdas dan IQ mereka lebih tinggi dari 110. Bisa 140 atau 150 tapi jarang.
     * Juga ada beberapa orang yang memiliki kecerdasan rendah dan IQ mereka lebih rendah dari 110. Bisa 40 atau 50 tapi jarang.
     * Dari informasi ini kita dapat mengatakan bahwa rata-rata IQ adalah 110. Dan katakanlah standar deviasi adalah 20.
     * Rata-rata dan simpangan baku adalah parameter distribusi normal.
     * Mari kita buat 100000 sampel dan visualisasikan dengan histogram.


In [ ]:
# parameters of normal distribution
mu, sigma = 110, 20  # mean and standard deviation
## membuat nilai random
s = np.random.normal(mu, sigma, 100000)
print("mean: ", np.mean(s))
print("standart deviation: ", np.std(s))
# visualize with histogram
plt.figure(figsize = (10,7))
plt.hist(s, 100, density=False)
plt.ylabel("frequency")
plt.xlabel("IQ")
plt.title("Histogram of IQ")
plt.show()

* Seperti yang dapat dilihat dari histogram sebagian besar orang terakumulasi mendekati 110 yang merupakan rata-rata dari distribusi normal kita
* Namun apa yang "paling" yang saya sebutkan di kalimat sebelumnya? Bagaimana jika saya ingin tahu berapa persentase orang yang harus memiliki skor IQ antara 80 dan 140?
* Kami akan menggunakan z-score untuk menjawab pertanyaan ini.
     * z = (x - rata-rata)/std
     * z1 = (80-110)/20 = -1.5
     * z2 = (140-110)/20 = 1,5
     * Jarak antara mean dan 80 adalah 1,5std dan jarak antara mean dan 140 adalah 1,5std.
     * Jika Anda melihat tabel z, Anda akan melihat bahwa 1,5std sesuai dengan 0,4332
 <a href="https://ibb.co/hys6OT"><img src="https://preview.ibb.co/fYzWq8/123.png" alt="123" border="0"></a>
     
     
     * Mari kita hitung dengan 2 karena 1 dari 80 berarti dan lainnya dari rata-rata hingga 140
     * 0,4332 * 2 = 0,8664
     * 86.64% orang memiliki IQ antara 80 dan 140.
     
  <a href="https://ibb.co/fhc6OT"><img src="https://preview.ibb.co/bUi2xo/hist.png" alt="hist" border="0"></a>

Berapa persentase orang yang seharusnya memiliki skor IQ kurang dari 80?
* z = (110-80)/20 = 1,5
* Mari kita lihat tabel skor z 0,4332. 43,32% orang memiliki IQ antara 80 dan mean(110).
* Jika kita kurangi dari 50% menjadi 43,32%, kita dapat menemukan persentase orang yang memiliki skor IQ kurang dari 80.
* 50-43,32 = 6,68. Hasil dari, **6,68% orang memiliki skor IQ kurang dari 80.**

## Jawaban